In [1]:
import os
import numpy as np
import pandas as pd
import open3d as o3d
from matplotlib import pyplot as plt

import loadlib
from data import KITTIPointCloud
from helpers.data_loaders import *
from helpers.pointcloud import filter_points

Using TensorFlow backend.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
PATH = '..'

In [5]:
training = dict(add_geometrical_features=True,
                subsample_flag=False,
                subsample_ratio=1,
                compute_eigen=0,
#                      model_name='lodnn'
                    )
kpc = KITTIPointCloud(feature_parameters=training, is_training=True, path='../../', view='front', dataset='semantickitti')

In [6]:
f_train, gt_train, f_valid, gt_valid = kpc.load_dataset(limit_index=10, classes=[40])

100%|██████████| 10/10 [00:00<00:00, 1276.88it/s]

Loading dataset
nCPUs = 8


-22.108658 3.0193346
-22.108658 3.0193346
nCPUs = 8


100%|██████████| 10/10 [00:00<00:00, 764.91it/s]

nCPUs = 8


-15.019754 3.0900204
-22.108658 3.0900204
nCPUs = 8
Height ranges from -22.108657836914062 to 3.0900204181671143
nCPUs = 8
nCPUs = 8


In [ ]:
idx = np.random.randint(len(f_train))

# titles = ['count', 
#           'mean_refl', 
#           'max_z', 
#           'min_z', 
#           'mean_z', 
#           'std_z', 
#           'nx', 
#           'ny',
#           'nz',
#           'curvature',
#           'eigenentropy',
#           'linearity',
#           'omnivariance',
#           'planarity',
#           'sphericity']

titles = ['rho',
          'reflectance',
          'z',
          'nx', 
          'ny',
          'nz',
          'curvature',
          'eigenentropy',
          'linearity',
          'omnivariance',
          'planarity',
          'sphericity']
fig, axs = plt.subplots(12, figsize=(15,15))
for i, ax in enumerate(axs.ravel()):
    im = ax.imshow(f_train[idx, :, :, i], cmap=plt.cm.coolwarm)
    ax.set_title(titles[i])
    fig.colorbar(im, ax=ax)
plt.show()

# plt.figure(figsize=(8,4))
# plt.imshow(np.abs(gt_train[idx, :, :, 0]))
# plt.show()

# plt.figure(figsize=(12,4))
# plt.imshow(np.abs(f_train[idx, :, :, -3:]))
# plt.show()



In [ ]:
gt_train.shape

In [ ]:
L = [[i1, i2] for i1, i2 in zip(kpc.train_set['pc'], kpc.train_set['labels'])]

In [ ]:
import keras
keras.utils.to_categorical(gt_train[idx, :, :, 0], num_classes=34)

In [ ]:

    
print(len(config['labels']))

In [ ]:
# np.array(list(config['labels'].keys()))
import yaml
with open('../source/semantic-kitti-api/config/semantic-kitti.yaml') as f:
    config = yaml.safe_load(f)
    
label_map = np.zeros(max(config['labels'].keys())+ 1)
for n, k in enumerate(config['labels'].keys()):
    label_map[k] = n

In [ ]:
keras.utils.to_categorical(label_map[gt_train[0,:,:,0].astype(int)], num_classes=34).shape

In [ ]:
f_train.max(axis=0).max(axis=0).max(axis=0), f_train.min(axis=0).min(axis=0).min(axis=0)

In [ ]:
from source.utils import KittiPointCloudClass as KPC

In [ ]:
features_flag = dict(add_geometrical_features=True,
                     subsample_flag=False,
                     subsample_ratio=1,
                     compute_HOG=False,
                     compute_eigen=100,
                     model_name='lodnn'
                    )
kpcc = KPC(features_flag, view='bev', dataset='kitti')

In [ ]:
f_old_train, gt_old_train, f_old_valid, gt_old_valid = kpcc.get_dataset()

In [ ]:
print(idx)
titles = ['count', 
          'mean_refl', 
          'max_z', 
          'min_z', 
          'mean_z', 
          'std_z', 
          'nx', 
          'ny',
          'nz',
          'curvature',
          'eigenentropy',
          'linearity',
          'omnivariance',
          'planarity',
          'sphericity']

fig, axs = plt.subplots(5, 3, figsize=(15,15))
for i, ax in enumerate(axs.ravel()):
    im = ax.imshow(f_old_train[idx, :, :, i]-f_train[idx, :, :, i])
    print(np.abs(f_old_train[idx, :, :, i]-f_train[idx, :, :,i]).sum())
    ax.set_title(titles[i])
    fig.colorbar(im, ax=ax)
plt.show()

In [ ]:
print(idx)
titles = ['count', 
          'mean_refl', 
          'max_z', 
          'min_z', 
          'mean_z', 
          'std_z', 
          'nx', 
          'ny',
          'nz',
          'curvature',
          'eigenentropy',
          'linearity',
          'omnivariance',
          'planarity',
          'sphericity']

fig, axs = plt.subplots(5, 3, figsize=(15,15))
for i, ax in enumerate(axs.ravel()):
    im = ax.imshow(f_old_train[idx, :, :, i])
#     print(np.abs(f_old_train[idx, :, :, i]-f_train[idx, :, :,i]).sum())
    ax.set_title(titles[i])
    fig.colorbar(im, ax=ax)
plt.show()

# plt.figure(figsize=(4,8))
# plt.imshow(np.abs(f_old_train[idx, :, :, -3:]))
# plt.show()

In [ ]:
cloud = load_pyntcloud(kpc.train_set['pc'][idx])
points = filter_points(cloud.points.values, side_range=(-10,10), fwd_range=(6, 46))
cloud = PyntCloud(pd.DataFrame(points, columns=['x', 'y', 'z', 'i']))

In [ ]:
cloud.points['curvature'] = kpc.proj.back_project(cloud.xyz, f_old_train[idx, :, :, -6])
cloud.points['eigentropy'] = kpc.proj.back_project(cloud.xyz, f_old_train[idx, :, :, -5])
cloud.points['linearity'] = kpc.proj.back_project(cloud.xyz, f_old_train[idx, :, :, -4])
cloud.points['omnivariance'] = kpc.proj.back_project(cloud.xyz, f_old_train[idx, :, :, -3])
cloud.points['planarity'] = kpc.proj.back_project(cloud.xyz, f_old_train[idx, :, :, -2])
cloud.points['sphericity'] = kpc.proj.back_project(cloud.xyz, f_old_train[idx, :, :, -1])

In [ ]:
cloud.plot(initial_point_size=0.03, use_as_color='planarity', cmap=plt.cm.coolwarm)

In [ ]:
k_neighs = cloud.get_neighbors(k=30)
eigenvalues = cloud.add_scalar_field("eigen_values", k_neighbors=k_neighs)
print(eigenvalues)
cloud.add_scalar_field("planarity", ev=eigenvalues)
#         k_neighbors = pc.get_neighbors(k=self.kneighbors)
#         eigenvalues = pc.add_scalar_field("eigen_values", k_neighbors=k_neighbors)
#         # anisotropy = pc.add_scalar_field("anisotropy", ev=eigenvalues)
#         curvature = pc.add_scalar_field("curvature", ev=eigenvalues)
#         eigenentropy = pc.add_scalar_field("eigenentropy", ev=eigenvalues)
#         # eigensum = pc.add_scalar_field("eigen_sum", ev=eigenvalues)
#         linearity = pc.add_scalar_field("linearity", ev=eigenvalues)
#         omnivariance = pc.add_scalar_field("omnivariance", ev=eigenvalues)
#         planarity = pc.add_scalar_field("planarity", ev=eigenvalues)
#         sphericity = pc.add_scalar_field("sphericity", ev=eigenvalues)

In [ ]:
cloud.points[eigenvalues].values.sum(axis=1)

In [ ]:
cloud.points[eigenvalues] = cloud.points[eigenvalues].values / cloud.points[eigenvalues].values.sum(axis=1)[:, None]

In [ ]:
cloud.points[eigenvalues].sum(axis=1).values[:, None]

In [ ]:
lambda_1 = cloud.points['e1(31)']
lambda_2 = cloud.points['e2(31)']
lambda_3 = cloud.points['e3(31)']
eig = cloud.points['eigenentropy(31)']

In [ ]:
lambda_sum = lambda_1 + lambda_2 + lambda_3
lambda_1 /= lambda_sum
lambda_2 /= lambda_sum
lambda_3 /= lambda_sum

In [ ]:
eig = -lambda_1 * np.log(lambda_1) -lambda_2 * np.log(lambda_2) -lambda_3 * np.log(lambda_3)
plan = (cloud.points['e2(31)'] - cloud.points['e3(31)'])/ cloud.points['e1(31)']

In [ ]:
cloud.points['eig'] = eig
cloud.points['plan'] = plan

In [ ]:
o3d_cloud = o3d.geometry.PointCloud()
o3d_cloud.points = o3d.utility.Vector3dVector(points[:,:3])
o3d.geometry.PointCloud.compute_mean_and_covariance(o3d_cloud.select_down_sample(np.arange(2)))

In [ ]:
kdflann = o3d.geometry.KDTreeFlann(o3d_cloud)

In [ ]:
all_Eig = np.zeros_like(points[:,:3])
for i in range(len(points)):
    n_neigh, ind, dists = kdflann.search_knn_vector_3d(cloud.xyz[i], 100)
    mean, cov = o3d.geometry.PointCloud.compute_mean_and_covariance(o3d_cloud.select_down_sample(ind))
    eigs = np.linalg.eigvals(cov) 
    eigs = np.sort(eigs)[::-1]
    eigs /= eigs.sum()
    all_Eig[i] = eigs

In [ ]:
cloud.points['eig']=(-all_Eig * np.log(all_Eig)).sum(axis=1)
cloud.points['plan'] = (all_Eig[:,2] - all_Eig[:,1]) / all_Eig[:,0]
cloud.points['omni'] = np.prod(all_Eig, axis=1)**(1/3)

In [ ]:
cloud.plot(initial_point_size=0.03, use_as_color='omni', cmap=plt.cm.coolwarm)

In [ ]:
np.prod(all_Eig, axis=1)**(1/3)

In [ ]:
np.prod(np.repeat(np.arange(3), 3000).reshape(3,3000).T)

In [ ]:
np.prod(np.repeat(np.arange(3)+1, 3000).reshape(3,3000).T,axis=1)